# Profiling main skforecast classes and methods

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [16]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset

%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [70]:
n = 5000
n_series = 500
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
# exog = [exog_i.copy().sample(frac=0.8).sort_index() for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}

value
series_id  datetime         
series_1   2023-01-01     67
           2023-01-02     51
           2023-01-03      1
           2023-01-04     82
           2023-01-05     66
...                      ...
series_499 2036-09-04     57
           2036-09-05     71
           2036-09-06     43
           2036-09-07     51
           2036-09-08     39

[2495000 rows x 1 columns]

exog_1  exog_2  exog_3  exog_4  exog_5  exog_6
series_id  datetime                                                  
series_1   2023-01-01       2       3       7       1       7       8
           2023-01-02       1       5       8       1       1       3
           2023-01-03       2       2       1       4       7       7
           2023-01-04       6       4       5       8       8       7
           2023-01-05       5       1       8       1       1       7
...                       ...     ...     ...     ...     ...     ...
series_499 2036-09-04       6       5       2       1       2       2
           2036-09-05       6       9       9       8       6       6
           2036-09-06       6       2       3       8       8       9
           2036-09-07       3       3       6       5       9       6
           2036-09-08       6       1       9       9       9       4

[2495000 rows x 6 columns]

In [44]:
exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
exog

[            exog_1  exog_2  exog_3  exog_4  exog_5  exog_6 series_id
 datetime                                                            
 2023-01-01       9       5       2       2       7       5  series_1
 2023-01-02       6       5       1       5       9       3  series_1
 2023-01-03       3       1       6       5       8       6  series_1
 2023-01-04       2       6       8       9       2       9  series_1
 2023-01-05       3       6       9       6       5       1  series_1
 ...            ...     ...     ...     ...     ...     ...       ...
 2036-09-04       9       9       9       1       2       4  series_1
 2036-09-05       6       8       5       6       2       6  series_1
 2036-09-06       1       2       1       9       6       1  series_1
 2036-09-07       2       1       9       3       6       9  series_1
 2036-09-08       4       2       3       7       3       7  series_1
 
 [5000 rows x 7 columns],
             exog_1  exog_2  exog_3  exog_4  exog_5  exog_6 se

In [40]:
df = series.copy()
df_2 = series.copy()
first_col = 'value'

In [41]:
df_2.index = df_2.index.set_names([df_2.index.names[0], None])

In [33]:
%%timeit

df.rename_axis(index={"datetime": None})

1.88 ms ± 49.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [34]:
%%timeit

df_2.index.set_names([df_2.index.names[0], None])

15.1 μs ± 248 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [36]:
exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2022-01-01", periods=n, freq="D"),
)

In [33]:
%%timeit

exog.reindex(
                                   series_dict[k].index, 
                                   fill_value = np.nan
                               )

245 μs ± 12.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [41]:
exog_empt = exog.loc[:'1900-01-01']

In [46]:
%%timeit

exog_empt.reindex(
    series_dict[k].index, 
    fill_value = np.nan
)

186 μs ± 15.9 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [54]:
%%timeit

pd.DataFrame(
    np.nan,
    columns=exog_empt.columns,
    index=series_dict[k].index,
    dtype=exog_empt.dtypes.to_numpy()
)

252 μs ± 8.72 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [53]:
exog_empt.dtypes.to_numpy()

array([dtype('int32'), dtype('int32'), dtype('int32'), dtype('int32'),
       dtype('int32'), dtype('int32')], dtype=object)

In [29]:
series_dict

{'series_1': datetime
 2023-01-01    50
 2023-01-02    25
 2023-01-03    74
 2023-01-04    31
 2023-01-05    28
               ..
 2036-09-04    67
 2036-09-05    86
 2036-09-06    94
 2036-09-07    96
 2036-09-08    95
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_2': datetime
 2023-01-01    38
 2023-01-02    91
 2023-01-03    58
 2023-01-04     6
 2023-01-05     4
               ..
 2036-09-04    39
 2036-09-05    23
 2036-09-06    21
 2036-09-07    51
 2036-09-08    49
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_3': datetime
 2023-01-01    56
 2023-01-02    20
 2023-01-03    20
 2023-01-04    29
 2023-01-05    85
               ..
 2036-09-04    93
 2036-09-05    19
 2036-09-06    74
 2036-09-07    71
 2036-09-08    70
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_4': datetime
 2023-01-01    99
 2023-01-02    44
 2023-01-03    95
 2023-01-04    51
 2023-01-05    84
               ..
 2036-09-04    94
 2036-09-05    52
 2036-09-06    52
 20

In [4]:
series_no_idx = series.reset_index(["series_id", "datetime"])
series_no_idx

,series_id,datetime,value
0,series_1,2023-01-01,50
1,series_1,2023-01-02,25
2,series_1,2023-01-03,74
3,series_1,2023-01-04,31
4,series_1,2023-01-05,28
...,...,...,...
2494995,series_499,2036-09-04,67
2494996,series_499,2036-09-05,75
2494997,series_499,2036-09-06,31
2494998,series_499,2036-09-07,11


In [7]:
%%timeit

series_no_idx.groupby('series_id', observed=True).size()
series_no_idx.groupby('series_id', observed=True).size()

321 ms ± 6.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit

data_grouped = series.groupby('series_id', observed=True)
data_grouped.size()
data_grouped.size()

203 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit

not_valid_index = [
    1 
    for k, v in series_dict.items()
]

14.2 μs ± 347 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
_ = [v.index.freqstr for v in series_dict.values()]
type(_[0])

str

In [20]:
import pandas as pd

# Regular, con frecuencia
idx = pd.date_range('2024-01-01', periods=3, freq='D')
print(idx.freqstr)  # 'D'

# Irregular, sin frecuencia
idx2 = pd.DatetimeIndex(['2024-01-01', '2024-01-03', '2024-01-04'])
print(idx2.freq)      # None
print(idx2.freqstr)   # None

# Eliminar una fecha (pierde frecuencia)
idx3 = idx.delete(1)
print(idx3.freq)      # None
print(idx3.freqstr)   # None


D
None
None
None
None


In [22]:
pd.RangeIndex(start=0, stop=10, step=1, name='index', dtype='int64').freqstr

AttributeError: 'RangeIndex' object has no attribute 'freqstr'

In [23]:
exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

In [27]:
not isinstance(exog.index, pd.MultiIndex)

True

In [28]:
exog.index.names

FrozenList([None])

In [26]:
not exog.index.names == ['series_id', 'datetime']

True

In [ ]:
%%timeit

freqs = [s.index.freq for s in series.values() if s.index.freq is not None]
if not freqs:
    raise ValueError("At least one series must have a frequency.")
if not all(f == freqs[0] for f in freqs):
    raise ValueError(
        "All series with frequency must have the same frequency."
    )

min_index = min([v.index[0] for v in series.values() if not v.empty])
max_index = max([v.index[-1] for v in series.values() if not v.empty])
span_index = pd.date_range(start=min_index, end=max_index, freq=freqs[0])

In [20]:
%%timeit

min_index = None
max_index = None
freq = None

for s in series_dict.values():
    if s.empty:
        continue
    # Fijar la frecuencia del primero que tenga
    if freq is None and s.index.freq is not None:
        freq = s.index.freq
    # Comparar min y max en una sola pasada
    first, last = s.index[0], s.index[-1]
    min_index = first if min_index is None else min(min_index, first)
    max_index = last if max_index is None else max(max_index, last)

if freq is None:
    raise ValueError("At least one series must have a frequency.")
span_index = pd.date_range(start=min_index, end=max_index, freq=freq)


3.43 ms ± 102 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit

freqs = [s.index.freq for s in series_dict.values() if s.index.freq is not None]
if not freqs:
    raise ValueError("At least one series must have a frequency.")
if not all(f == freqs[0] for f in freqs):
    raise ValueError(
        "All series with frequency must have the same frequency."
    )

min_index = min([v.index[0] for v in series_dict.values() if not v.empty])
max_index = max([v.index[-1] for v in series_dict.values() if not v.empty])
span_index = pd.date_range(start=min_index, end=max_index, freq=freqs[0])

4.32 ms ± 179 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
series_dict_2 = series_dict.copy()
series_dict_2['series_1'] = series_dict_2['series_1'].asfreq(None)

In [31]:
%%timeit

indexes_freq = set()
min_index = []
max_index = []
for v in series_dict_2.values():
    if v.empty:
        continue
    indexes_freq.add(v.index.freqstr)
    min_index.append(v.index[0])
    max_index.append(v.index[-1])

if not len(indexes_freq) == 1 or indexes_freq == {None}:
    raise ValueError(
        f"If `series` is a dictionary, all series must have a Pandas "
        f"DatetimeIndex as index with the same frequency. "
        f"Found frequencies: {sorted(indexes_freq)}"
    )
span_index = pd.date_range(
    start=min(min_index), end=max(max_index), freq=indexes_freq.pop()
)

3.98 ms ± 28.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%%timeit

indexes_freq = set()
min_index = []
max_index = []
for v in series_dict_2.values():
    if v.empty:
        continue
    idx = v.index
    indexes_freq.add(idx.freqstr)
    min_index.append(idx[0])
    max_index.append(idx[-1])

if not len(indexes_freq) == 1 or indexes_freq == {None}:
    raise ValueError(
        f"If `series` is a dictionary, all series must have a Pandas "
        f"DatetimeIndex as index with the same frequency. "
        f"Found frequencies: {sorted(indexes_freq)}"
    )
span_index = pd.date_range(
    start=min(min_index), end=max(max_index), freq=indexes_freq.pop()
)

3.94 ms ± 70.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [80]:
n = 10000
exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.nan
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

In [94]:
%%timeit

pd.DataFrame(
            data    = np.nan,
            columns = ['_dummy_exog_col_to_keep_shape'],
            index   = exog.index
         )


68.6 μs ± 7.35 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [95]:
%%timeit

pd.Series(
            data    = np.nan,
            name = '_dummy_exog_col_to_keep_shape',
            index   = exog.index
         )

26.2 μs ± 1.58 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [96]:
%%timeit

pd.Series(
            index   = exog.index,
            name = '_dummy_exog_col_to_keep_shape',
        )

41.7 μs ± 5.1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [98]:
exog_3 = pd.Series(
            data    = np.nan,
            index   = exog.index,
            name = '_dummy_exog_col_to_keep_shape'
        )

pd.concat([exog_3, exog], axis=0)

,_dummy_exog_col_to_keep_shape,exog_1,exog_2,exog_3,exog_4,exog_5,exog_6
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2050-05-14,NaN,2.0,6.0,1.0,2.0,2.0,NaN
2050-05-15,NaN,1.0,8.0,4.0,2.0,3.0,NaN
2050-05-16,NaN,7.0,8.0,7.0,4.0,6.0,NaN
2050-05-17,NaN,3.0,5.0,5.0,7.0,5.0,NaN


In [83]:
%%timeit

pd.concat([exog_3, exog], axis=0)

854 μs ± 104 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [84]:
%%timeit

pd.concat([exog_2, exog], axis=0)

972 μs ± 48.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [109]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10)), 
                       'l2': pd.Series(np.arange(10))})
series.index = pd.date_range(start='1-1-2018', periods=len(series), freq='D')
series

,l1,l2
2018-01-01,0,0
2018-01-02,1,1
2018-01-03,2,2
2018-01-04,3,3
2018-01-05,4,4
2018-01-06,5,5
2018-01-07,6,6
2018-01-08,7,7
2018-01-09,8,8
2018-01-10,9,9


In [113]:
data = series.copy()
freq = data.index.freqstr
data.index.name = "datetime"
data = data.reset_index()
data = pd.melt(data, id_vars="datetime", var_name="series_id", value_name="value")
data = data.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq(freq), include_groups=False
)
data.index.levels[1]

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10'],
              dtype='datetime64[ns]', name='datetime', freq='D')

In [106]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10)), 
                       'l2': pd.Series(np.arange(10))})
series.index = pd.date_range(start='1-1-2018', periods=len(series), freq='D')
series

,l1,l2
2018-01-01,0,0
2018-01-02,1,1
2018-01-03,2,2
2018-01-04,3,3
2018-01-05,4,4
2018-01-06,5,5
2018-01-07,6,6
2018-01-08,7,7
2018-01-09,8,8
2018-01-10,9,9


In [107]:
df = (
    series.melt(var_name="level", value_name="pred", ignore_index=False)
    .reset_index()
    .sort_values(by=["index", "level"])
    .set_index("index")
    .rename_axis(None, axis=0)
)
df

,level,pred
2018-01-01,l1,0
2018-01-01,l2,0
2018-01-02,l1,1
2018-01-02,l2,1
2018-01-03,l1,2
2018-01-03,l2,2
2018-01-04,l1,3
2018-01-04,l2,3
2018-01-05,l1,4
2018-01-05,l2,4


In [103]:
from skforecast.utils import preprocess_y

_, series_index = preprocess_y(y=series, return_values=False)
series = series.copy()
series.index = series_index
series_dict = series.to_dict("series")
series_dict

{'l1': 2018-01-01    0
 2018-01-02    1
 2018-01-03    2
 2018-01-04    3
 2018-01-05    4
 2018-01-06    5
 2018-01-07    6
 2018-01-08    7
 2018-01-09    8
 2018-01-10    9
 Freq: D, Name: l1, dtype: int64,
 'l2': 2018-01-01    0
 2018-01-02    1
 2018-01-03    2
 2018-01-04    3
 2018-01-05    4
 2018-01-06    5
 2018-01-07    6
 2018-01-08    7
 2018-01-09    8
 2018-01-10    9
 Freq: D, Name: l2, dtype: int64}

In [126]:
n = 500
n_series = 50
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    # index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

# freq = series.index.freq
series = pd.melt(series, id_vars=None, var_name="series_id", value_name="value")
series.index.name = "datetime"
series = series.reset_index().set_index(["series_id", "datetime"])
series

value
series_id datetime       
series_1  0            84
          1            92
          2            97
          3             4
          4            62
...                   ...
series_49 24495        17
          24496        46
          24497        55
          24498        93
          24499        95

[24500 rows x 1 columns]

In [138]:
n = 500
n_series = 50
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    # index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
series = pd.melt(series, id_vars=None, var_name="series_id", value_name="value")
series = series.groupby("series_id", sort=False).apply(
    lambda g: g.set_index(pd.RangeIndex(len(g)), append=False).rename_axis('idx', axis=0),
    include_groups=False
)
series

value
series_id idx       
series_1  0       84
          1       86
          2       57
          3       83
          4       40
...              ...
series_49 495     61
          496     16
          497     72
          498     28
          499     51

[24500 rows x 1 columns]

In [141]:
series.loc['series_1'].index.step

AttributeError: 'Index' object has no attribute 'step'

In [145]:
n = 500
n_series = 50
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    # index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
series = pd.melt(series, id_vars=None, var_name="series_id", value_name="value")
series.index.name = "datetime"
series = series.reset_index().set_index(["series_id", "datetime"])
series_grouped = series.groupby(level=0, group_keys=False, sort=False)
series = series_grouped.apply(
    lambda g: g.set_index(
        pd.MultiIndex.from_arrays(
            [g.index.get_level_values(0), pd.RangeIndex(len(g))],
            names=g.index.names
        )
    )
)  
series

value
series_id datetime       
series_1  0            17
          1            94
          2            81
          3            17
          4            53
...                   ...
series_49 495          34
          496          24
          497          67
          498          63
          499          96

[24500 rows x 1 columns]

In [146]:
series.loc['series_1'].index

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       490, 491, 492, 493, 494, 495, 496, 497, 498, 499],
      dtype='int64', name='datetime', length=500)

In [80]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

In [81]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
}

In [83]:
forecaster = ForecasterRecursiveMultiSeries(regressor=Ridge(random_state=123), 
                                                          lags=2, transformer_series=None,
                                                          encoding='onehot')
forecaster.fit(series=series_dict)
forecaster.predict(steps=5, levels=None)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,pred
50,series_1,49.999406
50,series_2,39.999652
50,series_3,69.999396
51,series_1,50.999085
51,series_2,40.999454
51,series_3,70.999070
52,series_1,51.998603
52,series_2,41.999156
52,series_3,71.998580
53,series_1,52.998177


In [84]:
forecaster = ForecasterRecursiveMultiSeries(regressor=Ridge(random_state=123), 
                                                          lags=2, transformer_series=None,
                                                          encoding='onehot')

cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,window_size=forecaster.window_size
         )
cv.split(X=series_dict,as_pandas=True)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

Information of folds
--------------------
Number of observations used for initial training: 30
Number of observations used for backtesting: 20
    Number of folds: 7
    Number skipped folds: 0 
    Number of steps per fold: 3
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 2 observations.

Fold: 0
    Training:   0 -- 29  (n=30)
    Validation: 30 -- 32  (n=3)
Fold: 1
    Training:   No training in this fold
    Validation: 33 -- 35  (n=3)
Fold: 2
    Training:   No training in this fold
    Validation: 36 -- 38  (n=3)
Fold: 3
    Training:   No training in this fold
    Validation: 39 -- 41  (n=3)
Fold: 4
    Training:   No training in this fold
    Validation: 42 -- 44  (n=3)
Fold: 5
    Training:   No training in this fold
    Validation: 45 -- 47  (n=3)
Fold: 6
    Training:   No training in this fold
    Validation: 48 -- 49  (n=2)



,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,30,28,30,30,33,30,33,True
1,1,0,30,31,33,33,36,33,36,False
2,2,0,30,34,36,36,39,36,39,False
3,3,0,30,37,39,39,42,39,42,False
4,4,0,30,40,42,42,45,42,45,False
5,5,0,30,43,45,45,48,45,48,False
6,6,0,30,46,48,48,50,48,50,False


In [94]:
cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,
            window_size=forecaster.window_size, 
            verbose=False
         )
metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster            = forecaster,
                                            series                = series_dict,
                                            cv                    = cv,
                                            levels                = 'l1',
                                            metric                = 'mean_absolute_error',
                                            add_aggregated_metric = False,
                                            exog                  = None,
                                            verbose               = False,
                                            n_jobs=1
                                        )

backtest_predictions.head(3)

ValueError: Levels ['l1'] not found in `series`, available levels are ['series_1', 'series_2', 'series_3']. Review `levels` argument.

In [36]:
type(series_dict['series_1'].index)

pandas.core.indexes.range.RangeIndex

In [37]:
isinstance(series_dict['series_1'].index, type(series_dict['series_1'].index))

True

In [38]:
from skforecast.utils import check_preprocess_series



In [42]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
}

In [63]:
from skforecast.preprocessing import series_wide_to_long

series = pd.DataFrame({'1': pd.Series(np.arange(10, dtype=float)), 
                           '2': pd.Series(np.arange(10, dtype=float))})
series.index = pd.date_range(start='2000-01-01', periods=len(series), freq='D')
series = series_wide_to_long(series)

exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})
exog.index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, 3)]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])

In [65]:
forecaster = ForecasterRecursiveMultiSeries(
        Ridge(), lags=5, encoding='ordinal_category', transformer_series=None
    )

results = forecaster._create_train_X_y(
    series=series, exog=exog, store_last_window=['3']
) 

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── MissingExogWarning ─────────────────────────────────╮
│ {'1', '2'} not present in `exog`. All values of the exogenous variables for these    │
│ series will be NaN.                                                                  │
│                                                                                      │
│ Category : MissingExogWarning                                                        │
│ Location : <string>:51                                                               │
│ Suppress : warnings.simplefilter('ignore', category=MissingExogWarning)              │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── IgnoredArgumentWarning ───────────────────────────────╮
│ Series {'3'} are not present in `series`. No last window is stored for them.         │
│                                                                                      │
│ Category : IgnoredArgumentWarning                                                    │
│ Location : <string>:165                                                              │
│ Suppress : warnings.simplefilter('ignore', category=IgnoredArgumentWarning)          │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [66]:
results[0]

,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
2000-01-06,4.0,3.0,2.0,1.0,0.0,0
2000-01-07,5.0,4.0,3.0,2.0,1.0,0
2000-01-08,6.0,5.0,4.0,3.0,2.0,0
2000-01-09,7.0,6.0,5.0,4.0,3.0,0
2000-01-10,8.0,7.0,6.0,5.0,4.0,0
2000-01-06,4.0,3.0,2.0,1.0,0.0,1
2000-01-07,5.0,4.0,3.0,2.0,1.0,1
2000-01-08,6.0,5.0,4.0,3.0,2.0,1
2000-01-09,7.0,6.0,5.0,4.0,3.0,1
2000-01-10,8.0,7.0,6.0,5.0,4.0,1


In [77]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_4': None
}

In [78]:
cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,window_size=forecaster.window_size, verbose=False
         )

In [79]:
cv._extract_index(X=series_dict)

AttributeError: 'NoneType' object has no attribute 'index'

In [ ]:
not_valid_index = []
indexes_freq = set()
min_index = []
max_index = []
for k, v in series_dict.items():
    if v.empty:
        continue
    idx = v.index
    if isinstance(v.index, pd.DatetimeIndex):
        indexes_freq.add(v.index.freqstr)
    elif isinstance(v.index, pd.RangeIndex):
        indexes_freq.add(v.index.step)
    else:
        not_valid_index.append(k)

    min_index.append(idx[0])
    max_index.append(idx[-1])

if not_valid_index:
    raise TypeError(
        f"If `series` is a dictionary, all series must have a Pandas "
        f"RangeIndex or DatetimeIndex with the same step/frequency. "
        f"Review series: {not_valid_index}"
    )

if not len(indexes_freq) == 1 or indexes_freq == {None}:
    raise ValueError(
        f"If `series` is a dictionary, all series must have a Pandas "
        f"RangeIndex or DatetimeIndex with the same step/frequency. "
        f"Found frequencies: {sorted(indexes_freq)}"
    )

indexes_freq

6.96 ms ± 547 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
pd.RangeIndex(start=min(min_index), stop=max(max_index) + 1, step=2)

RangeIndex(start=0, stop=50, step=2)

In [ ]:
%%timeit

not_valid_index = []
indexes_freq = set()
min_index = []
max_index = []
for k, v in series_dict.items():
    if v.empty:
        continue
    idx = v.index
    if isinstance(v.index, pd.DatetimeIndex):
        indexes_freq.add(v.index.freqstr)
    elif isinstance(v.index, pd.RangeIndex):
        indexes_freq.add(v.index.step)
    else:
        not_valid_index.append(k)

    min_index.append(idx[0])
    max_index.append(idx[-1])

if not_valid_index:
    raise TypeError(
        f"If `series` is a dictionary, all series must have a Pandas "
        f"RangeIndex or DatetimeIndex with the same step/frequency. "
        f"Review series: {not_valid_index}"
    )

if not len(indexes_freq) == 1 or indexes_freq == {None}:
    raise ValueError(
        f"If `series` is a dictionary, all series must have a Pandas "
        f"RangeIndex or DatetimeIndex with the same step/frequency. "
        f"Found frequencies: {sorted(indexes_freq)}"
    )

indexes_freq

In [76]:
_ = pd.Series(np.arange(10), index=pd.date_range(start='2023-01-01', periods=10, freq='D'))
type(_.loc['2024-01-01':].index)

pandas.core.indexes.datetimes.DatetimeIndex

In [90]:
series = pd.DataFrame({'1': pd.Series(np.arange(10, dtype=float)), 
                           '2': pd.Series(np.arange(10, dtype=float))})

In [91]:
['l1'].issubset(series.columns)

AttributeError: 'list' object has no attribute 'issubset'